In [ ]:
# default_exp basic_model
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import GPT2Config, GPT2LMHeadModel
from kirby.run_params import RunParams
from kirby.data_manager import DataManager

# Basic Model

> A basic Pytorch lighting module

In [ ]:
#export
class BasicModel(pl.LightningModule):
    def __init__(self, run_params):
        super().__init__()
        self.run_params = run_params
        config = GPT2Config()
        self.model = GPT2LMHeadModel(config)
        self.loss = torch.nn.CrossEntropyLoss(reduction='none')
        pass

    def prepare_data(self):
        data_manager = DataManager(self.run_params)
        self.train_ds, self.val_ds = data_manager.prepare_data()


    def forward(self, x):
        loss = self.model(x['input_ids'], attention_mask=x['attention_mask'], labels=x['labels'])[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.forward(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.forward(batch)
        self.log('val_loss', loss)
        return loss

    def validation_epoch_end(self, losses):
        loss = torch.cat([loss.unsqueeze(0) for loss in losses], 0).mean()
        self.log('val_loss', loss)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
                self.train_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=True,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                ) 

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
                self.val_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=False,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                )

    def configure_optimizers(self):
        return torch.optim.SGD(
            self.parameters(),
            lr=self.run_params.lr,
            momentum=self.run_params.momentum,
        ) 

# Testing

In [ ]:
# Creation
basic_model = BasicModel(RunParams())
assert isinstance(basic_model, BasicModel)

In [ ]:
run_params = RunParams(debug=True)
basic_model = BasicModel(run_params)
trainer = pl.Trainer(
    default_root_dir='logs',
    gpus=(1 if torch.cuda.is_available() else 0),
    max_epochs=run_params.max_epochs,
    fast_dev_run=run_params.debug,
    logger=TensorBoardLogger(save_dir='logs/', name=run_params.run_name),
)

trainer.fit(basic_model)

In [ ]:
basic_model.train_ds